## 作業
在鐵達尼資料集中，   
可以用今天範例中提到的三種辨識異常值的方法，  
以 training data 為資料集。  
1.觀察票價是否有異常值的現象?  
2.你覺得找出的異常是真的異常? 你覺得需要做處理嗎?  
(試著講出自己的想法。)  


### 資料
鐵達尼號 train 資料集


In [1]:
# library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn  as sns
from scipy import stats
import math
import statistics
#讓圖可以在 jupyter notebook顯示
%matplotlib inline

In [2]:
df = pd.read_csv("Titanic_train.csv")

### 探索性分析:
*  觀察-是否有異常值  
我們來觀察一下 Fare 變數吧!!

## Q1:觀察票價是否有異常值的現象?

## 方法1
* 進行一個描述性的統計（規定範圍），從而檢視哪些值是不合理的（範圍以外的值）。  


In [4]:
print(df["Fare"].isnull().sum(), "筆缺失值") 
print(df["Fare"].describe()) #最小值0(是免費嗎?!)，最大值512，落差這麼大!!而平均數32
print("票價為0的", df["Fare"].apply(lambda x : x==0).sum(), "筆") 

0 筆缺失值
count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64
票價為0的 15 筆


## 方法 2
* 進行3倍標準差原則的計算，從而檢視哪些值是可疑的異常值。  


In [7]:
def outer_z_score(data, sigma=3):
    mean_data = np.mean(data)
    std_data = np.std(data)
    z_score = list((x-mean_data)/std_data for x in data)
    return np.where(np.abs(z_score)>sigma)

outer_index = outer_z_score(df["Fare"])
#print(outer_index) 
print(len(outer_index[0]), "筆疑似是異常值") #20筆
print(df.loc[outer_index[0], "Fare"])

20 筆疑似是異常值
27     263.0000
88     263.0000
118    247.5208
258    512.3292
299    247.5208
311    262.3750
341    263.0000
377    211.5000
380    227.5250
438    263.0000
527    221.7792
557    227.5250
679    512.3292
689    211.3375
700    227.5250
716    227.5250
730    211.3375
737    512.3292
742    262.3750
779    211.3375
Name: Fare, dtype: float64


## 方法3-盒鬚圖判別法(IQR method)

In [8]:
def outer_iqr(data, d=1.5):
    q1, q3 = np.percentile(df["Fare"], [25, 75])
    #print(q1, q3)
    iqr = q3-q1    
    lower_bound = q1-iqr*d
    upper_bound = q3+iqr*d
    return np.where((data>upper_bound)|(data<lower_bound))

outer_index2 = outer_iqr(df["Fare"], 2)
#print(outer_index2) 
print(len(outer_index2[0]), "筆疑似是異常值") 
print(df.loc[outer_index2[0], "Fare"])

92 筆疑似是異常值
27     263.0000
31     146.5208
34      82.1708
61      80.0000
62      83.4750
         ...   
829     80.0000
835     83.1583
849     89.1042
856    164.8667
879     83.1583
Name: Fare, Length: 92, dtype: float64


## Q2:你覺得找出的異常是真的異常? 你覺得需要做處理嗎?
